# Day 6 - Lab 2: Creating a Conversational Multi-Agent System (Solution)

**Objective:** Integrate the multi-agent LangGraph system from the previous lab into the FastAPI backend, creating a new `/chat` endpoint that is stateful and can handle conversational memory.

**Introduction:**
This solution notebook provides the complete code for deploying the LangGraph agent as a conversational API. It covers creating a stateless endpoint, building a Streamlit UI, and finally implementing stateful memory management.

## Step 1: Setup

**Explanation:**
This setup code mocks the necessary components for the lab. We create a mock `multi_agent_app` to simulate the compiled LangGraph from Lab 6.1. This allows us to develop the API and UI without needing to run the full, complex graph. We also initialize an in-memory dictionary, `conversation_histories`, which will serve as our simple session store for the advanced challenge.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uuid
import streamlit as st
import requests
from utils import save_artifact

app = FastAPI()

class MockLangGraphApp:
    def invoke(self, inputs, config):
        session_id = config.get('configurable', {}).get('session_id', 'N/A')
        print(f"Invoking agent for session {session_id}")
        question = inputs.get('question')
        # In a real app, LangGraph's memory would use the session_id to get history
        return {"final_answer": f"This is a stateful answer to: '{question}'"}

multi_agent_app = MockLangGraphApp()

# In-memory store for conversation histories (for demonstration)
conversation_histories = {}

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Creating a Stateless Chat Endpoint

**Explanation:**
This is the most basic form of an API endpoint. It defines a `ChatRequest` model to validate the incoming data, accepts a `POST` request, and simply invokes the agent with the question. It has no concept of memory; every call is treated as a brand new conversation.

In [ ]:
class ChatRequest(BaseModel):
    question: str

@app.post("/chat")
def chat_endpoint(request: ChatRequest):
    # Note the empty config dictionary, indicating a stateless call
    result = multi_agent_app.invoke({"question": request.question}, config={})
    return {"answer": result.get("final_answer")}

### Challenge 2 (Intermediate): Building a Simple Streamlit UI

**Explanation:**
This script creates a simple web UI. Streamlit's `st.session_state` is used to maintain state across user interactions within the app, allowing us to build up a chat history for display. When the user clicks the button, it uses the popular `requests` library to send the question to our running FastAPI backend and displays the JSON response.

In [ ]:
chat_ui_code = """
import streamlit as st
import requests

st.title("Multi-Agent RAG Chatbot")

if 'history' not in st.session_state:
    st.session_state.history = []

question = st.text_input("Ask a question about the project:", key="input")

if st.button("Send"):
    if question:
        # For this intermediate step, we call the stateless endpoint
        response = requests.post("http://127.0.0.1:8000/chat", json={"question": question})
        if response.status_code == 200:
            answer = response.json().get('answer')
            st.session_state.history.append(("You", question))
            st.session_state.history.append(("Agent", answer))
        else:
            st.error("Failed to get response from API.")

for author, text in st.session_state.history:
    st.write(f"**{author}:** {text}")
"""
save_artifact(chat_ui_code, "labs/Day_06_RAG_Agents/chat_ui.py")
print("Saved 'chat_ui.py'. To run it, open your terminal and execute: streamlit run labs/Day_06_RAG_Agents/chat_ui.py")

### Challenge 3 (Advanced): Implementing Conversational Memory

**Explanation:**
This is the complete, stateful solution. 
1.  **`StatefulChatRequest`**: The request model now includes an optional `session_id`.
2.  **Endpoint Logic**: The endpoint checks if a `session_id` was provided. If not, it creates a new one, establishing a new conversation session.
3.  **`config` object**: This is the most critical part. We pass a `config` dictionary to the `.invoke()` method. LangGraph specifically looks for the `{"configurable": {"session_id": "..."}}` structure to manage memory. When it sees this, it automatically retrieves the history for that session, includes it in the context for the LLM, and saves the new turn to that same history.
4.  **UI Update**: The Streamlit UI must be updated to store the `session_id` it receives from the first response and send it back on all subsequent requests, maintaining the conversational context.

In [ ]:
class StatefulChatRequest(BaseModel):
    question: str
    session_id: str | None = None

@app.post("/stateful_chat")
def stateful_chat_endpoint(request: StatefulChatRequest):
    session_id = request.session_id if request.session_id else str(uuid.uuid4())
    
    # In a real app with LangGraph's built-in memory, you just need to pass the session_id in the config.
    # LangGraph handles the retrieval and saving of history for you.
    config = {"configurable": {"session_id": session_id}}
    
    inputs = {"question": request.question}
    result = multi_agent_app.invoke(inputs, config=config)
    
    return {
        "answer": result.get("final_answer"),
        "session_id": session_id
    }

stateful_chat_ui_code = """
import streamlit as st
import requests

st.title("STATEFUL Multi-Agent RAG Chatbot")

if 'history' not in st.session_state:
    st.session_state.history = []
if 'session_id' not in st.session_state:
    st.session_state.session_id = None

question = st.text_input("Ask a follow-up question:", key="input_stateful")

if st.button("Send Stateful Request"):
    if question:
        payload = {"question": question, "session_id": st.session_state.session_id}
        response = requests.post("http://127.0.0.1:8000/stateful_chat", json=payload)
        if response.status_code == 200:
            data = response.json()
            st.session_state.session_id = data.get('session_id') # Store the session ID
            answer = data.get('answer')
            st.session_state.history.append(("You", question))
            st.session_state.history.append(("Agent", answer))
        else:
            st.error("Failed to get response from API.")

for author, text in st.session_state.history:
    st.write(f"**{author}:** {text}")
st.write(f"Current Session ID: {st.session_state.session_id}")
"""
save_artifact(stateful_chat_ui_code, "stateful_chat_ui.py")
print("Saved 'stateful_chat_ui.py'. Update your UI code and run it to test the stateful endpoint.")

## Lab Conclusion

Excellent! You have successfully integrated your powerful multi-agent system into a real-world application. You created a stateless API endpoint, built a UI for it, and then performed the critical upgrade to make it a stateful, conversational agent with memory. This is the complete pattern for deploying sophisticated AI assistants that can engage in natural, multi-turn dialogues with users.